# Subindo o modelo
No terminal do conda com seu environment ativo, digite: mlflow models serve -m runs:/<run_id>/<model_name>


## XGBoost

No meu caso aqui vai ser mlflow models serve -m runs:/1507bc65599b4d31bb4f60bd130b3ade/XGBoost

## Keras estruturado

No meu caso aqui vai ser mlflow models serve -m runs:/1507bc65599b4d31bb4f60bd130b3ade/XGBoost

## Keras não estruturado

No meu caso aqui vai ser mlflow models serve -m runs:/1507bc65599b4d31bb4f60bd130b3ade/XGBoost

# Fazendo requisicoes para os modelos

Os dados precisam estar em uma de 4 formas possíveis

**Forma 1**:
  ```json
    {
      "dataframe_split": {
          "columns": ["a", "b", "c"],
          "data": [[1, 2, 3], [4, 5, 6]]
      }
    }
  ```
   
**Forma 2**:
  ```json
    {
      "dataframe_records": [
        {"a": 1,"b": 2,"c": 3},
        {"a": 4,"b": 5,"c": 6}
      ]
    }
  ```
**Forma 3**:
  ```json
    {
      "instances": [
          {"a": "s1", "b": 1, "c": [1, 2, 3]},
          {"a": "s2", "b": 2, "c": [4, 5, 6]},
          {"a": "s3", "b": 3, "c": [7, 8, 9]}
      ]
    }
  ```
**Forma 4**:
  ```json
    {
      "inputs": {"question": ["What color is it?"],
                "context": ["Some people said it was green but I know that it is pink."]},
      "params": {"max_answer_len": 10}
    }
  ```

In [1]:
import requests
import json
import tensorflow as tf
import numpy as np

## XGBoost

### Aqui será usada a forma 1

In [17]:
dados = {
    'columns':['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal'],
    'data':[[ 62. ,   1. ,   1. , 120. , 281. ,   0. ,   0. , 103. ,   0. ,
         1.4,   1. ,   1. ,   3. ]]   
}

In [18]:
dataframe_split = {'dataframe_split':dados}

In [19]:
dados_json = json.dumps(dataframe_split)

In [20]:
previsao = requests.post(url='http://localhost:5001/invocations', 
                         headers={'Content-Type' : 'application/json' , }, data=dados_json)

In [21]:
previsao.text

'{"predictions": [0]}'

## Keras estruturado

### Aqui será usada a forma 1

In [14]:
dados = {
    'columns':['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal'],
    'data':[[ 62. ,   1. ,   1. , 120. , 281. ,   0. ,   0. , 103. ,   0. ,
         1.4,   1. ,   1. ,   3. ]]   
}

In [15]:
dataframe_split = {'dataframe_split':dados}

In [16]:
dados_json = json.dumps(dataframe_split)

In [17]:
previsao = requests.post(url='http://localhost:5001/invocations', 
                         headers={'Content-Type' : 'application/json' , }, data=dados_json)

In [18]:
previsao.text

'{"predictions": [{"0": 0.2638627886772156}]}'

## Keras não estruturado

### Aqui será usada a forma 4

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
m = np.max(x_train)
x_train = x_train/m
x_test = x_test/m
x_test = np.expand_dims(x_test,axis=-1)

In [3]:
dados = [x_test[0].tolist(),x_test[1].tolist()]

In [5]:
inputs = {"instances":dados}

In [6]:
inputs_json = json.dumps(inputs)

In [7]:
previsao = requests.post(url='http://localhost:5001/invocations', 
                         headers={'Content-Type' : 'application/json'}, data=inputs_json)

In [8]:
previsao.text

'{"predictions": [[2.5471723347436637e-05, 2.5978971862627986e-08, 1.5691597582190298e-05, 0.00037669745506718755, 5.5897126003401354e-05, 2.4391134502366185e-05, 8.72670327112246e-08, 0.996576726436615, 0.00013180830865167081, 0.0027930845972150564], [0.001173503464087844, 0.00012517916911747307, 0.9791313409805298, 0.0028648499865084887, 6.740989988429646e-08, 0.004117809701710939, 0.008625703863799572, 2.0426994817857036e-10, 0.003961524460464716, 1.5547259479831155e-08]]}'

In [9]:
previsoes = json.loads(previsao.text)

In [10]:
array_resposta = np.array(previsoes['predictions'][0])

In [11]:
array_resposta.argmax(0)

7